In [1]:
import pandas as pd
import os

In [2]:
current_path = os.getcwd()
print(current_path)

c:\Users\sepujas\Dev\mat\notebooks


In [3]:
base_path = os.path.abspath(os.path.join(current_path, os.pardir))
print(base_path)

c:\Users\sepujas\Dev\mat


In [6]:
zle00_serial_stock_file_path = os.path.join(base_path, 'data', 'raw', 'tbl_zle00_serial_stock.txt')
print(zle00_serial_stock_file_path)
zle00_serial_stock_preprocessed_path = os.path.join(base_path, 'data', 'preprocessed', 'tbl_zle00_serial_stock_file_path.csv')
print(zle00_serial_stock_preprocessed_path)

c:\Users\sepujas\Dev\mat\data\raw\tbl_zle00_serial_stock.txt
c:\Users\sepujas\Dev\mat\data\preprocessed\tbl_zle00_serial_stock_file_path.csv


In [7]:
df_zle00_serial_stock = pd.read_csv(zle00_serial_stock_file_path, sep='\t', skiprows=3, encoding='latin1')

In [8]:
df_zle00_serial_stock.head()

,Unnamed: 0,SOrg.,Plnt,Name 1,SLoc,Description,Material,Material Description,Profl.,Serial Number,Unnamed: 10,PP,Equipment,Pstng Date,Total,Serials,BUn
0,NaN,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8001351,NaN,1,300029830821,20.01.2010,NaN,4,PC
1,NaN,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8001502,NaN,1,300031076094,21.06.2010,NaN,4,PC
2,NaN,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8002519,NaN,1,300034116235,28.02.2011,NaN,4,PC
3,NaN,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8002627,NaN,1,300034116236,28.02.2011,NaN,4,PC
4,NaN,3000,3090,Hilti TS Mexico,100,3090-0100,274935,Diamond coring tool DD 120 120V,Z001,254782,NaN,1,300024180201,28.05.2008,NaN,2,PC


In [9]:
unnamed_columns = [col for col in df_zle00_serial_stock.columns if 'Unnamed:' in col]
df_zle00_serial_stock.drop(columns=unnamed_columns, inplace=True)
column_titles = df_zle00_serial_stock.columns.tolist()
new_column_titles = {col: col.strip().replace(' ', '_').replace('-','_').replace('.','') for col in column_titles}
df_zle00_serial_stock.rename(columns=new_column_titles, inplace=True)

In [10]:
column_counts = df_zle00_serial_stock.columns.value_counts()
duplicate_columns = column_counts[column_counts > 1].index

In [11]:
for col in duplicate_columns:
    col_indices = [i for i, x in enumerate(df_zle00_serial_stock.columns) if x == col]
    for j, index in enumerate(col_indices):
        df_zle00_serial_stock.columns.values[index] = f"{col}_{j+1}"

In [12]:
df_zle00_serial_stock.columns.to_list()

['SOrg',
 'Plnt',
 'Name_1',
 'SLoc',
 'Description',
 'Material',
 'Material_Description',
 'Profl',
 'Serial_Number',
 'PP',
 'Equipment',
 'Pstng_Date',
 'Total',
 'Serials',
 'BUn']

In [13]:
df_zle00_serial_stock.head()

,SOrg,Plnt,Name_1,SLoc,Description,Material,Material_Description,Profl,Serial_Number,PP,Equipment,Pstng_Date,Total,Serials,BUn
0,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8001351,1,300029830821,20.01.2010,NaN,4,PC
1,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8001502,1,300031076094,21.06.2010,NaN,4,PC
2,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8002519,1,300034116235,28.02.2011,NaN,4,PC
3,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8002627,1,300034116236,28.02.2011,NaN,4,PC
4,3000,3090,Hilti TS Mexico,100,3090-0100,274935,Diamond coring tool DD 120 120V,Z001,254782,1,300024180201,28.05.2008,NaN,2,PC


In [14]:
df_zle00_serial_stock.dtypes

SOrg                      int64
Plnt                      int64
Name_1                   object
SLoc                      int64
Description              object
Material                  int64
Material_Description     object
Profl                    object
Serial_Number            object
PP                        int64
Equipment                 int64
Pstng_Date               object
Total                   float64
Serials                   int64
BUn                      object
dtype: object

In [16]:
def transform_columns(df):
    str_columns = [
        'Name_1', 'Description', 'Material_Description' ,'Profl', 'Serial_Number', 'BUn']
    for col in str_columns:
        df[col] = df[col].astype(str).str.strip()

    numeric_columns = ['SOrg', 'Plnt', 'SLoc', 'Material', 'PP', 'Equipment', 'Total' , 'Serials']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int).astype(str)

    date_columns = ['Pstng_Date']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors="coerce", format='%d.%m.%Y')

transform_columns(df_zle00_serial_stock)

In [17]:
df_zle00_serial_stock.head()

,SOrg,Plnt,Name_1,SLoc,Description,Material,Material_Description,Profl,Serial_Number,PP,Equipment,Pstng_Date,Total,Serials,BUn
0,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8001351,1,300029830821,2010-01-20,0,4,PC
1,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8001502,1,300031076094,2010-06-21,0,4,PC
2,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8002519,1,300034116235,2011-02-28,0,4,PC
3,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8002627,1,300034116236,2011-02-28,0,4,PC
4,3000,3090,Hilti TS Mexico,100,3090-0100,274935,Diamond coring tool DD 120 120V,Z001,254782,1,300024180201,2008-05-28,0,2,PC


In [19]:
df_zle00_serial_stock['key_plant'] = df_zle00_serial_stock['Plnt'] + '/' + df_zle00_serial_stock['Material']
df_zle00_serial_stock['key_plant'] = df_zle00_serial_stock['key_plant'].astype(str).str.strip()
df_zle00_serial_stock['key_material'] = df_zle00_serial_stock['SOrg'] + '/' + df_zle00_serial_stock['Material']
df_zle00_serial_stock['key_material'] = df_zle00_serial_stock['key_material'].astype(str).str.strip()

In [20]:
df_zle00_serial_stock.head()

,SOrg,Plnt,Name_1,SLoc,Description,Material,Material_Description,Profl,Serial_Number,PP,Equipment,Pstng_Date,Total,Serials,BUn,key_plant,key_material
0,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8001351,1,300029830821,2010-01-20,0,4,PC,3090/272441,3000/272441
1,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8001502,1,300031076094,2010-06-21,0,4,PC,3090/272441,3000/272441
2,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8002519,1,300034116235,2011-02-28,0,4,PC,3090/272441,3000/272441
3,3000,3090,Hilti TS Mexico,100,3090-0100,272441,Battery charger C 4/36-ACS Li-Ion 115V,Z001,8002627,1,300034116236,2011-02-28,0,4,PC,3090/272441,3000/272441
4,3000,3090,Hilti TS Mexico,100,3090-0100,274935,Diamond coring tool DD 120 120V,Z001,254782,1,300024180201,2008-05-28,0,2,PC,3090/274935,3000/274935


In [21]:
df_zle00_serial_stock.to_csv(zle00_serial_stock_preprocessed_path, index=False)
